In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Rescaling, GlobalAveragePooling2D, Input, Flatten, RandomBrightness, RandomContrast, Dropout, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
IMG_SIZE = 180

In [3]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [4]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", batch_size=32, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 200 images belonging to 8 classes.


In [5]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", batch_size=32, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 40 images belonging to 8 classes.


In [6]:
train_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [7]:
test_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [8]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

6

In [9]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

1

In [10]:
data_augmentation = Sequential(
    [

    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    # RandomContrast(0.2),
    # RandomBrightness(0.2)
    ]
)
data_augmentation

<Sequential name=sequential, built=False>

In [11]:
input_shape = IMG_SIZE, IMG_SIZE, 3
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,  
    weights='imagenet'
)
base_model.trainable = True

base_model.summary()

C:\Users\donof\AppData\Local\Temp\ipykernel_20552\4258986162.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 180, 180,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 90, 90,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 90, 90,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 90, 90,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 90, 90,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 90, 90,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 90, 90,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 90, 90,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 90, 90,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 91, 91,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 45, 45,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 45, 45,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 45, 45,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 45, 45,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [12]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [13]:
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    # Rescaling(1./255),
    base_model,
    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(train_set.class_indices))
    # Conv2D(16,(3,3), padding='same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Conv2D(64,(3,3), padding = 'same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Dropout(0.3),
    # Flatten(),
    # Dense(256, activation = 'relu'),
    # Dropout(0.4),
    # Dense(len(train_set.class_indices), activation='softmax')
])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46080)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 46080)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,898,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,157,384 (31.12 MB)

 Trainable params: 7,261,320 (27.70 MB)

 Non-trainable params: 896,064 (3.42 MB)

In [14]:
# for layer in base_model.layers[:-30]:
#     layer.trainable = False

In [15]:
opt = Adam(learning_rate=0.001)

callback = EarlyStopping(monitor='val_accuracy',
                         patience=5,
                         verbose=1,
                         mode="max",
                         baseline=0.3,
                          restore_best_weights=True,
                          start_from_epoch=10
                         )


In [16]:

model.compile(
            optimizer=opt,
              loss = CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=20,
                          #  steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                            validation_steps=STEP_SIZE_TEST,
                           verbose=1,
                           callbacks=[callback]
                           )


Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 915ms/step - accuracy: 0.2000 - loss: 6.3954 - val_accuracy: 0.1875 - val_loss: 14.9838
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 522ms/step - accuracy: 0.4400 - loss: 1.8221 - val_accuracy: 0.1562 - val_loss: 22.2215
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 512ms/step - accuracy: 0.5300 - loss: 1.6015 - val_accuracy: 0.1562 - val_loss: 25.5079
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 525ms/step - accuracy: 0.6150 - loss: 1.3880 - val_accuracy: 0.1875 - val_loss: 22.3699
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 536ms/step - accuracy: 0.7000 - loss: 0.9226 - val_accuracy: 0.2500 - val_loss: 20.6188
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 514ms/step - accuracy: 0.7800 - loss: 0.7717 - val_accuracy: 0.2500 - val_loss: 22.1030
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 571ms/step - accuracy: 0.7600 - loss: 0.8410 - val_accuracy: 0.2812 - val_loss: 24.1185
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 528ms/step - accuracy: 0.8350 - loss: 1.0581 - val_accuracy: 0.3438 - v

In [17]:
model.save("ttte_model.keras")